In [1]:
from neo4j import GraphDatabase

def get_neo4j_driver() -> GraphDatabase.driver:
    """Establishes and returns a Neo4j session for AuraDB."""
    neo4j_uri = 'neo4j+s://9d1381c2.databases.neo4j.io:7687'
    neo4j_user = 'Shirley'
    neo4j_password = 'Sxl19950312'
    return GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

# Call the function to get the driver
driver = get_neo4j_driver()

# Open a session
with driver.session() as session:
    # Verify connectivity within the session
    session.run("RETURN 1")

print("Connectivity verified successfully.")


from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd


# Define a Cypher query to retrieve the required data
query = """
MATCH (n)
RETURN n
"""

# Execute the query and process the results
with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]

# Converting Neo4j Data to Pandas DataFrame Format
df = pd.DataFrame(neo4j_data)

#print(df)

Connectivity verified successfully.


In [2]:
query = """
MATCH (p:Product)-[:HAS_RECIPE]->(i:Recipe)-[:USES_INGREDIENT]->(N:Ingredient)-[:HAS_RESEARCH]->(a:IngredientResearch)
RETURN p.name AS Product, a.ingredient_name AS IngredientResearch
"""


with driver.session() as session:
    result = session.run(query)
    data = [record.data() for record in result]

df = pd.DataFrame(data)

fig = px.scatter(df, x='Product', y='IngredientResearch', color='IngredientResearch',
                 title='Ingredient in Products',
                 labels={'IngredientResearch': 'Ingredients'})

fig.update_layout(
    xaxis_title='Product',
    yaxis_title='Ingredients'
)

fig.show()

In [3]:
fig = px.bar(df, x='Product', y='IngredientResearch', color='IngredientResearch',
             title='Ingredients in Products',
             labels={'IngredientResearch': ' Ingredients'})

fig.update_layout(
    xaxis_title='Product',
    yaxis_title='Ingredients',
    xaxis_showticklabels=False,  # hide x label
    yaxis_showticklabels=False  # hide x label
)

fig.show()

In [ ]:
import plotly.graph_objects as go
from neo4j import GraphDatabase

query = """
MATCH (p:Product)
RETURN p.name AS Product, p.cupPrice AS CupPrice, p.price AS Price
"""

def get_product_prices():
    products = []
    cup_prices = []
    prices = []
    with driver.session() as session:
        result = session.run(query)
        for record in result:
            product = record["Product"]
            cup_price = record["CupPrice"]
            price = record["Price"]
            if cup_price is not None and price is not None:
                products.append(product)
                cup_prices.append(cup_price)
                prices.append(price)
    return products, cup_prices, prices

products, cup_prices, prices = get_product_prices()

fig = go.Figure(data=[
    go.Bar(name='Cup Price', x=products, y=cup_prices),
    go.Bar(name='Price', x=products, y=prices)
])

# Change layout
fig.update_layout(
    barmode='group',
    title="Product Cup Price and Price Comparison",
    xaxis_title="Product",
    yaxis_title="Value"
)

fig.show()




In [5]:
import plotly.express as px
import plotly.colors

# 获取唯一的成分列表
unique_ingredients = df['IngredientResearch'].unique()

# 生成足够的颜色以避免重叠
color_palette = plotly.colors.qualitative.Alphabet + plotly.colors.qualitative.Dark2 + plotly.colors.qualitative.Set1
num_colors = len(unique_ingredients)
extended_color_palette = color_palette * (num_colors // len(color_palette) + 1)
color_discrete_map = {unique_ingredients[i]: extended_color_palette[i] for i in range(num_colors)}

# 创建柱状图
fig = px.bar(df, x='Product', y='IngredientResearch', color='IngredientResearch',
             title='Ingredients in Products',
             labels={'IngredientResearch': ' Ingredients'},
             color_discrete_map=color_discrete_map)

fig.update_layout(
    xaxis_title='Product',
    yaxis_title='Ingredients',
    xaxis_showticklabels=False,  # 隐藏x轴标签
    yaxis_showticklabels=False   # 隐藏y轴标签
)

fig.show()
